In [42]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# 1. Load CIFAR-100 data
(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='fine')

# 2. One-hot encode the labels
enc = OneHotEncoder(sparse_output=False)
y_train = enc.fit_transform(y_train)
y_test = enc.transform(y_test)

# 3. Define preprocessing function (resize + normalize)
def preprocess(image, label):
    image = tf.image.resize(image, [128, 128])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# 4. Shuffle training data before splitting
train_indices = np.arange(len(X_train))
np.random.shuffle(train_indices)

X_train = X_train[train_indices]
y_train = y_train[train_indices]

# 5. Compute split sizes
val_size = int(0.1 * len(X_train))
X_val = X_train[:val_size]
y_val = y_train[:val_size]
X_train = X_train[val_size:]
y_train = y_train[val_size:]

# 6. Create tf.data.Dataset pipelines
batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=1000)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)


In [43]:
import tensorflow as tf
from tensorflow.keras import layers, models

# from preprocess import get_cifar100_datasets

# train_ds, val_ds, test_ds = get_cifar100_datasets(image_size=(128, 128), batch_size=32)

In [44]:
def conv_block(x, growth_rate):
    x1 = layers.BatchNormalization()(x)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(4 * growth_rate, (1, 1), use_bias=False)(x1)

    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(growth_rate, (3, 3), padding='same', use_bias=False)(x1)

    x = layers.Concatenate()([x, x1])
    return x

def dense_block(x, layers_count, growth_rate):
    for _ in range(layers_count):
        x = conv_block(x, growth_rate)
    return x

def transition_layer(x, compression=0.5):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(reduced_filters, (1, 1), use_bias=False)(x)
    x = layers.AveragePooling2D((2, 2), strides=2)(x)
    return x

def build_densenet121(input_shape=(128, 128, 3), num_classes=100):
    growth_rate = 32
    block_layers = [6, 12, 24, 16]

    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (7, 7), strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)

    for i, num_layers in enumerate(block_layers):
        x = dense_block(x, num_layers, growth_rate)
        if i != len(block_layers) - 1:
            x = transition_layer(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs, name="DenseNet121_Custom")
    return model


In [45]:
model = build_densenet121()

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "DenseNet121_Custom"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_960 (Conv2D)             (None, 64, 64, 64)   9408        input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_968 (BatchN (None, 64, 64, 64)   256         conv2d_960[0][0]                 
__________________________________________________________________________________________________
activation_968 (Activation)     (None, 64, 64, 64)   0           batch_normalization_968[0][0]    
_________________________________________________________________________________

In [46]:
model.fit(
    train_dataset,
    epochs=4,
    validation_data=val_dataset,
    # callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
    # verbose=1
)

Epoch 1/4
2813/2813 [==============================] - 225s 70ms/step - loss: 3.9243 - accuracy: 0.1006 - val_loss: 3.5661 - val_accuracy: 0.1682
Epoch 2/4
 154/2813 [>.............................] - ETA: 2:50 - loss: 3.4315 - accuracy: 0.1765

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=1)
print(f'Test accuracy: {test_accuracy:.4f}')